In [2]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import time
import sys

#Set device
USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    device = torch.device("cuda")
    cuda = True
else:
    device = torch.device("cpu")
    cuda = False
    
print("Device =", device)
gpus = [0]

Device = cuda


In [3]:
def time_elapsed(start_time):
    elapsed = time.time() - start_time
    hours = int(elapsed/3600)
    minutes = int(int(elapsed/60)%60)
    seconds = int(elapsed%60)
    
    return hours, minutes, seconds

In [4]:
#Load SMILES data as integer labels and as one-hot encoding
data = np.load("ohesmiles.npz")
data = data["arr_0"]

intdata = np.load("intsmiles.npz")
intdata = intdata["arr_0"]

data = torch.from_numpy(data).view(np.shape(data)[0], 1, np.shape(data)[1])
intdata = torch.from_numpy(intdata)

print("Dataset size: " + str(data.size()))
print("Integer dataset size: " + str(intdata.size()))

Dataset size: torch.Size([20066538, 1, 53])
Integer dataset size: torch.Size([20066538])


In [5]:
#Get input tensor
def inptarg(i, batch_size):
    while True:
        try:
            
            #Define input and target tensor sizes
            inp = torch.Tensor((seq_length - 1) * batch_size, 1, np.shape(data)[2])
            target = torch.Tensor((seq_length - 1) * batch_size)
    
            #SMILES molecules in batch 
            inputs = data[int((i * seq_length)) : int((i * seq_length) + (seq_length * batch_size)), :, :]
            targets = intdata[int((i * seq_length)) : int((i * seq_length) + (seq_length * batch_size))]

            #Index counters for input, target
            r = 0
            s = 0 
            for p in range(seq_length * batch_size - 1):
        
                if (p % seq_length != (seq_length - 1)):
                    #Input data (does not include last character in SMILES)
                    inp[r, :, :] = inputs[p, :, :]
                    r += 1
            
                if (p % seq_length != 0):
                    #Target data (does not include first character in SMILES)
                    target[s] = targets[p]
                    s += 1
                    
            return inp, target
        except:
            continue
        break

In [6]:
#Define model
class Model(nn.Module):
    
    #Define model parameters
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(Model, self).__init__()
        
        #Model parameters
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout
        
        #Model layers
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, dropout = dropout)
        self.linear = nn.Linear(hidden_size, input_size)
        
    #Define initial hidden and cell states
    def init_states(self, num_layers, hidden_size):
        hidden = [Variable(torch.zeros(num_layers, 1, hidden_size)),
                  Variable(torch.zeros(num_layers, 1, hidden_size))]
        
        return hidden
    
    #Define forward propagation
    def forward(self, inp, hidden):
        #LSTM
        output, hidden = self.lstm(inp, hidden)

        #Linear Layer
        output = self.linear(output)
        
        return output, hidden
    

In [14]:
#Define training
def train(epochs):
    #Set start time
    start_time = time.time()
    
    #Does not support shuffling of the data (converges well within an epoch)
    #Iterate over desired number of epochs 
    for e in range(epochs):
        
        #Iterate set of seq_length characters
        for i in range(int((np.shape(data)[0] - seq_length + 1) / batch_size)):
            
            #Initialize hidden and cell states
            hidden = model.init_states(num_layers, hidden_size)
            
            #Run on GPU if available
            if cuda:
                hidden = (hidden[0].cuda(), hidden[1].cuda())
        
            #Set initial gradients
            model.zero_grad()
    
            #Set initial loss
            loss = 0 
            
            #Get input and target
            input_data, target_data = inptarg(i, batch_size)
            input_data = input_data.float()
            target_data = target_data.long()
            
            #Run on GPU if available
            if cuda:
                input_data = input_data.cuda()
                target_data = target_data.cuda()
                
            #Run model, calculate loss
            output, hidden = model(input_data, hidden)
            loss += criterion(output.squeeze(), target_data.squeeze())
                
            #Backpropagate loss
            loss.backward()
            optimizer.step()
            
            #Update list of losses
            if (e == i == 0):
                losses[0] = loss.data.item() / seq_length   
            losses.append(loss.data.item() / seq_length)
            
            if (i % 100 == 0):
                torch.save(model.state_dict(), "test.pth")
            
            #Print training info
            hours, minutes, seconds = time_elapsed(start_time)
            print("Loss: {:0.6f}".format(loss.data.item() / seq_length) + " | ΔLossTotal: {:+0.4f}".format(losses[-1] - losses[1]) + " | Epoch: {0:02d}".format(e + 1) + " | Iteration: {0:04d}".format(i + 1) + " | Time elapsed: {0:02d}".format(hours) + "h {0:02d}".format(minutes) + " m {0:02d}".format(seconds) + " s")

In [15]:
#Initialize model parameters
input_size = np.shape(data)[2]
hidden_size = 1024
num_layers = 3
dropout = .2
learning_rate = 0.001
epochs = 1
seq_length = 75
batch_size = 64

#List of losses
losses = [0]

In [16]:
#Call model, set optimizer and loss function
model = Model(input_size, hidden_size, num_layers, dropout)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()

#Run on GPU if available
if cuda:
    model.cuda()
    criterion.cuda()  

In [17]:
#Total number of parameters
total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters in network: " + str(total_params))

Total number of parameters in network: 21267509


In [ ]:
#Train
train(epochs)

Loss: 0.052871 | ΔLossTotal: +0.0000 | Epoch: 01 | Iteration: 0001 | Time elapsed: 00h 00 m 02 s
Loss: 0.051620 | ΔLossTotal: -0.0013 | Epoch: 01 | Iteration: 0002 | Time elapsed: 00h 00 m 04 s
Loss: 0.044088 | ΔLossTotal: -0.0088 | Epoch: 01 | Iteration: 0003 | Time elapsed: 00h 00 m 06 s
Loss: 0.048208 | ΔLossTotal: -0.0047 | Epoch: 01 | Iteration: 0004 | Time elapsed: 00h 00 m 08 s
Loss: 0.038468 | ΔLossTotal: -0.0144 | Epoch: 01 | Iteration: 0005 | Time elapsed: 00h 00 m 10 s
Loss: 0.038136 | ΔLossTotal: -0.0147 | Epoch: 01 | Iteration: 0006 | Time elapsed: 00h 00 m 13 s
Loss: 0.038988 | ΔLossTotal: -0.0139 | Epoch: 01 | Iteration: 0007 | Time elapsed: 00h 00 m 15 s
Loss: 0.038753 | ΔLossTotal: -0.0141 | Epoch: 01 | Iteration: 0008 | Time elapsed: 00h 00 m 17 s
Loss: 0.037935 | ΔLossTotal: -0.0149 | Epoch: 01 | Iteration: 0009 | Time elapsed: 00h 00 m 19 s
Loss: 0.037022 | ΔLossTotal: -0.0158 | Epoch: 01 | Iteration: 0010 | Time elapsed: 00h 00 m 21 s
Loss: 0.036327 | ΔLossTotal: -

Loss: 0.035678 | ΔLossTotal: -0.0172 | Epoch: 01 | Iteration: 0086 | Time elapsed: 00h 03 m 02 s
Loss: 0.035619 | ΔLossTotal: -0.0173 | Epoch: 01 | Iteration: 0087 | Time elapsed: 00h 03 m 04 s
Loss: 0.035625 | ΔLossTotal: -0.0172 | Epoch: 01 | Iteration: 0088 | Time elapsed: 00h 03 m 06 s
Loss: 0.035632 | ΔLossTotal: -0.0172 | Epoch: 01 | Iteration: 0089 | Time elapsed: 00h 03 m 08 s
Loss: 0.035570 | ΔLossTotal: -0.0173 | Epoch: 01 | Iteration: 0090 | Time elapsed: 00h 03 m 10 s
Loss: 0.035604 | ΔLossTotal: -0.0173 | Epoch: 01 | Iteration: 0091 | Time elapsed: 00h 03 m 13 s
Loss: 0.035581 | ΔLossTotal: -0.0173 | Epoch: 01 | Iteration: 0092 | Time elapsed: 00h 03 m 15 s
Loss: 0.035564 | ΔLossTotal: -0.0173 | Epoch: 01 | Iteration: 0093 | Time elapsed: 00h 03 m 17 s
Loss: 0.035583 | ΔLossTotal: -0.0173 | Epoch: 01 | Iteration: 0094 | Time elapsed: 00h 03 m 19 s
Loss: 0.035611 | ΔLossTotal: -0.0173 | Epoch: 01 | Iteration: 0095 | Time elapsed: 00h 03 m 21 s
Loss: 0.035566 | ΔLossTotal: -

Loss: 0.035842 | ΔLossTotal: -0.0170 | Epoch: 01 | Iteration: 0171 | Time elapsed: 00h 06 m 02 s
Loss: 0.035931 | ΔLossTotal: -0.0169 | Epoch: 01 | Iteration: 0172 | Time elapsed: 00h 06 m 04 s
Loss: 0.035989 | ΔLossTotal: -0.0169 | Epoch: 01 | Iteration: 0173 | Time elapsed: 00h 06 m 07 s
Loss: 0.035937 | ΔLossTotal: -0.0169 | Epoch: 01 | Iteration: 0174 | Time elapsed: 00h 06 m 09 s
Loss: 0.035917 | ΔLossTotal: -0.0170 | Epoch: 01 | Iteration: 0175 | Time elapsed: 00h 06 m 11 s
Loss: 0.035882 | ΔLossTotal: -0.0170 | Epoch: 01 | Iteration: 0176 | Time elapsed: 00h 06 m 13 s
Loss: 0.035819 | ΔLossTotal: -0.0171 | Epoch: 01 | Iteration: 0177 | Time elapsed: 00h 06 m 15 s
Loss: 0.035876 | ΔLossTotal: -0.0170 | Epoch: 01 | Iteration: 0178 | Time elapsed: 00h 06 m 17 s
Loss: 0.035861 | ΔLossTotal: -0.0170 | Epoch: 01 | Iteration: 0179 | Time elapsed: 00h 06 m 19 s
Loss: 0.035865 | ΔLossTotal: -0.0170 | Epoch: 01 | Iteration: 0180 | Time elapsed: 00h 06 m 21 s
Loss: 0.035806 | ΔLossTotal: -

Loss: 0.035932 | ΔLossTotal: -0.0169 | Epoch: 01 | Iteration: 0256 | Time elapsed: 00h 09 m 03 s
Loss: 0.035871 | ΔLossTotal: -0.0170 | Epoch: 01 | Iteration: 0257 | Time elapsed: 00h 09 m 05 s
Loss: 0.035880 | ΔLossTotal: -0.0170 | Epoch: 01 | Iteration: 0258 | Time elapsed: 00h 09 m 07 s
Loss: 0.035849 | ΔLossTotal: -0.0170 | Epoch: 01 | Iteration: 0259 | Time elapsed: 00h 09 m 09 s
Loss: 0.035780 | ΔLossTotal: -0.0171 | Epoch: 01 | Iteration: 0260 | Time elapsed: 00h 09 m 11 s


In [26]:
torch.save(model.state_dict(), "test.pth")